# 📊 NOTEBOOK 4 : EXPLORATION ET FEATURE ENGINEERING DES IMAGES

## Objectifs
- ✅ Analyse statistique complète (taille, qualité, couleurs)
- ✅ Détection d'images problématiques
- ✅ Feature engineering (HOG, couleurs, textures)
- ✅ Visualisation séparabilité classes (PCA, UMAP)
- ✅ Recommandations preprocessing

## Dataset
- 7 catégories produits e-commerce
- ~150 images/catégorie
- Total: ~1050 images

In [ ]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Computer Vision
import cv2
from PIL import Image
from skimage import feature
from skimage.feature import hog, local_binary_pattern

# Dimensionality reduction
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
import umap

# Utils
from tqdm import tqdm
import json

# Style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

print("✅ Imports réussis")

## 1. CONFIGURATION

In [ ]:
# Chemins - ADAPTER À VOTRE STRUCTURE
BASE_DIR = Path.cwd().parent
DATA_DIR = BASE_DIR / "Data"
IMAGES_DIR = DATA_DIR / "Images"
RESULTS_DIR = BASE_DIR / "results" / "exploration"
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

METADATA_FILE = DATA_DIR / "data_images_corrected.csv"

# Catégories
CATEGORIES = [
    "Baby Care",
    "Beauty and Personal Care",
    "Computers",
    "Home Decor & Festive Needs",
    "Home Furnishing",
    "Kitchen & Dining",
    "Watches"
]

print(f"📂 Images: {IMAGES_DIR}")
print(f"📂 Results: {RESULTS_DIR}")
print(f"\n📦 {len(CATEGORIES)} catégories")

## 2. CHARGEMENT DES DONNÉES

In [ ]:
# Charger métadonnées
df = pd.read_csv(METADATA_FILE)

print(f"✅ Dataset: {df.shape[0]} images")
print(f"\n📊 Distribution:")
print(df['main_category'].value_counts())

# Vérifier images présentes
valid_idx = []
for idx, row in tqdm(df.iterrows(), total=len(df), desc="Vérification"):
    if (IMAGES_DIR / row['image']).exists():
        valid_idx.append(idx)

df = df.loc[valid_idx].reset_index(drop=True)
print(f"\n✅ {len(df)} images valides")

## 3. ANALYSE STATISTIQUE

In [ ]:
def analyze_image(img_path):
    """Analyser propriétés d'une image"""
    try:
        # Taille
        with Image.open(img_path) as img:
            w, h = img.size
        
        # Qualité (OpenCV)
        img_gray = cv2.imread(str(img_path), cv2.IMREAD_GRAYSCALE)
        sharpness = cv2.Laplacian(img_gray, cv2.CV_64F).var()
        brightness = np.mean(img_gray)
        contrast = np.std(img_gray)
        
        # Couleurs
        img_rgb = cv2.cvtColor(cv2.imread(str(img_path)), cv2.COLOR_BGR2RGB)
        avg_color = np.mean(img_rgb, axis=(0,1))
        
        return {
            'width': w, 'height': h, 'aspect_ratio': w/h,
            'sharpness': sharpness, 'brightness': brightness, 'contrast': contrast,
            'avg_r': avg_color[0], 'avg_g': avg_color[1], 'avg_b': avg_color[2]
        }
    except:
        return None

# Analyser
stats = []
for idx, row in tqdm(df.iterrows(), total=len(df), desc="Analyse"):
    s = analyze_image(IMAGES_DIR / row['image'])
    if s:
        s['category'] = row['main_category']
        s['filename'] = row['image']
        stats.append(s)

stats_df = pd.DataFrame(stats)
print(f"\n✅ {len(stats_df)} images analysées")
print(f"\n{stats_df.describe()}")

### Visualisations des distributions

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(18, 10))

# Largeur
axes[0,0].hist(stats_df['width'], bins=30, color='skyblue', edgecolor='black')
axes[0,0].set_title('Largeurs', fontweight='bold')
axes[0,0].set_xlabel('Pixels')

# Hauteur
axes[0,1].hist(stats_df['height'], bins=30, color='lightcoral', edgecolor='black')
axes[0,1].set_title('Hauteurs', fontweight='bold')
axes[0,1].set_xlabel('Pixels')

# Aspect ratio
axes[0,2].hist(stats_df['aspect_ratio'], bins=30, color='lightgreen', edgecolor='black')
axes[0,2].set_title('Ratios d\'Aspect', fontweight='bold')

# Netteté
axes[1,0].hist(stats_df['sharpness'], bins=30, color='gold', edgecolor='black')
axes[1,0].axvline(100, color='red', linestyle='--', label='Seuil (100)')
axes[1,0].set_title('Netteté (Laplacian)', fontweight='bold')
axes[1,0].legend()

# Luminosité
axes[1,1].hist(stats_df['brightness'], bins=30, color='orange', edgecolor='black')
axes[1,1].axvline(50, color='red', linestyle='--', label='Min')
axes[1,1].axvline(200, color='red', linestyle='--', label='Max')
axes[1,1].set_title('Luminosité', fontweight='bold')
axes[1,1].legend()

# Contraste
axes[1,2].hist(stats_df['contrast'], bins=30, color='purple', edgecolor='black')
axes[1,2].axvline(30, color='red', linestyle='--', label='Seuil')
axes[1,2].set_title('Contraste', fontweight='bold')
axes[1,2].legend()

plt.tight_layout()
plt.savefig(RESULTS_DIR / 'distributions.png', dpi=300, bbox_inches='tight')
plt.show()

### Détection images problématiques

In [ ]:
# Seuils qualité (basés sur recherche)
stats_df['is_blurry'] = stats_df['sharpness'] < 50
stats_df['bad_exposure'] = (stats_df['brightness'] < 50) | (stats_df['brightness'] > 200)
stats_df['low_contrast'] = stats_df['contrast'] < 30

print("⚠️ IMAGES PROBLÉMATIQUES")
print(f"Floues: {stats_df['is_blurry'].sum()}")
print(f"Mauvaise exposition: {stats_df['bad_exposure'].sum()}")
print(f"Faible contraste: {stats_df['low_contrast'].sum()}")

if stats_df['is_blurry'].sum() > 0:
    print("\n📋 Exemples floues:")
    print(stats_df[stats_df['is_blurry']][['filename','category','sharpness']].head())

### Analyse par catégorie

In [ ]:
cat_stats = stats_df.groupby('category').agg({
    'sharpness': 'mean',
    'brightness': 'mean',
    'contrast': 'mean'
}).round(2)

print("📊 QUALITÉ PAR CATÉGORIE")
print(cat_stats)

# Graphique
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

cat_stats['sharpness'].plot(kind='barh', ax=axes[0], color='skyblue')
axes[0].set_title('Netteté par Catégorie', fontweight='bold')
axes[0].axvline(100, color='red', linestyle='--', alpha=0.5)

cat_stats['brightness'].plot(kind='barh', ax=axes[1], color='gold')
axes[1].set_title('Luminosité par Catégorie', fontweight='bold')

cat_stats['contrast'].plot(kind='barh', ax=axes[2], color='coral')
axes[2].set_title('Contraste par Catégorie', fontweight='bold')

plt.tight_layout()
plt.savefig(RESULTS_DIR / 'quality_by_category.png', dpi=300, bbox_inches='tight')
plt.show()

## 4. FEATURE ENGINEERING

In [ ]:
def extract_features(img_path, size=(128,128)):
    """Features: HOG + Couleurs + Textures"""
    try:
        img = cv2.imread(str(img_path))
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        img_resized = cv2.resize(img_rgb, size)
        gray_resized = cv2.resize(img_gray, size)
        
        # HOG (forme)
        hog_feat = feature.hog(
            gray_resized, orientations=9,
            pixels_per_cell=(8,8), cells_per_block=(2,2),
            feature_vector=True
        )[:200]  # Tronquer
        
        # Histogramme couleurs (3 canaux × 32 bins)
        color_hist = []
        for ch in range(3):
            hist = cv2.calcHist([img_resized], [ch], None, [32], [0,256])
            color_hist.extend((hist / hist.sum()).flatten())
        
        # LBP (texture)
        lbp = feature.local_binary_pattern(gray_resized, 8, 1, 'uniform')
        lbp_hist, _ = np.histogram(lbp.ravel(), bins=10, range=(0,10), density=True)
        
        # Moments couleur
        moments = []
        for ch in range(3):
            moments.append(np.mean(img_resized[:,:,ch]))
            moments.append(np.std(img_resized[:,:,ch]))
        
        return np.concatenate([hog_feat, color_hist, lbp_hist, moments])
    except:
        return None

# Extraction
print("🔧 Extraction features...")
features, labels, files = [], [], []

for idx, row in tqdm(df.iterrows(), total=len(df), desc="Features"):
    feat = extract_features(IMAGES_DIR / row['image'])
    if feat is not None:
        features.append(feat)
        labels.append(row['main_category'])
        files.append(row['image'])

X = np.array(features)
y = np.array(labels)

print(f"\n✅ Features: {X.shape}")

# Sauvegarder
np.save(RESULTS_DIR / 'features.npy', X)
np.save(RESULTS_DIR / 'labels.npy', y)
np.save(RESULTS_DIR / 'files.npy', np.array(files))

## 5. VISUALISATION SÉPARABILITÉ

### PCA

In [ ]:
print("📊 PCA...")

pca = PCA(n_components=2, random_state=42)
X_pca = pca.fit_transform(X)

print(f"Variance: {pca.explained_variance_ratio_.sum():.2%}")

# Visualisation
label_map = {cat: i for i, cat in enumerate(CATEGORIES)}
y_num = np.array([label_map[l] for l in y])

plt.figure(figsize=(14, 10))
scatter = plt.scatter(X_pca[:,0], X_pca[:,1], c=y_num, cmap='tab10', alpha=0.6, s=50)
plt.colorbar(scatter, label='Catégorie')
plt.xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.1%})', fontweight='bold')
plt.ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.1%})', fontweight='bold')
plt.title('PCA - Séparabilité des Classes', fontweight='bold', fontsize=14)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.savefig(RESULTS_DIR / 'pca.png', dpi=300, bbox_inches='tight')
plt.show()

### UMAP

In [ ]:
print("📊 UMAP...")

reducer = umap.UMAP(n_components=2, n_neighbors=15, min_dist=0.1, 
                    metric='cosine', random_state=42)
X_umap = reducer.fit_transform(X)

plt.figure(figsize=(14, 10))
scatter = plt.scatter(X_umap[:,0], X_umap[:,1], c=y_num, cmap='tab10', alpha=0.6, s=50)
plt.colorbar(scatter, label='Catégorie')
plt.xlabel('UMAP 1', fontweight='bold')
plt.ylabel('UMAP 2', fontweight='bold')
plt.title('UMAP - Séparabilité des Classes', fontweight='bold', fontsize=14)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.savefig(RESULTS_DIR / 'umap.png', dpi=300, bbox_inches='tight')
plt.show()

### Silhouette Score

In [ ]:
sil_pca = silhouette_score(X_pca, y_num)
sil_umap = silhouette_score(X_umap, y_num)

print("\n" + "="*70)
print("📊 SÉPARABILITÉ")
print("="*70)
print(f"Silhouette PCA:  {sil_pca:.3f}")
print(f"Silhouette UMAP: {sil_umap:.3f}")
print("\nInterprétation:")
print("  > 0.5 : Excellente séparation")
print("  0.3-0.5: Bonne séparation")
print("  < 0.3 : Faible séparation")

if sil_umap > 0.5:
    print("\n✅ Classes bien séparées !")
elif sil_umap > 0.3:
    print("\n✅ Séparation acceptable")
else:
    print("\n⚠️ Augmentation données critique")

## 6. RECOMMANDATIONS PREPROCESSING

In [ ]:
print("\n" + "="*70)
print("📋 RECOMMANDATIONS")
print("="*70)

median_w = stats_df['width'].median()
median_h = stats_df['height'].median()

print(f"\n1. REDIMENSIONNEMENT")
print(f"   Actuel: {median_w:.0f} × {median_h:.0f}")
print(f"   Recommandé: 224 × 224")
print(f"   Pourquoi: Standard Transfer Learning")

print(f"\n2. NORMALISATION")
print(f"   Mean: [0.485, 0.456, 0.406]")
print(f"   Std:  [0.229, 0.224, 0.225]")
print(f"   Pourquoi: Stats ImageNet")

print(f"\n3. AUGMENTATION (critique pour 150 img/classe)")
print(f"   - Rotation: ±15°")
print(f"   - Translation: ±20%")
print(f"   - Flip horizontal: 50%")
print(f"   - Zoom: ±20%")
print(f"   - Color jitter: ±20%")
print(f"   - MixUp alpha: 0.2")

n_prob = (stats_df['is_blurry'] | stats_df['bad_exposure']).sum()
print(f"\n4. IMAGES PROBLÉMATIQUES: {n_prob}")
if n_prob > 0:
    print(f"   → Inspecter et retirer si nécessaire")

## 7. SAUVEGARDE

In [ ]:
# Stats complètes
stats_df.to_csv(RESULTS_DIR / 'stats.csv', index=False)

# Embeddings
np.save(RESULTS_DIR / 'pca.npy', X_pca)
np.save(RESULTS_DIR / 'umap.npy', X_umap)

# Métriques
metrics = {
    'n_images': len(stats_df),
    'n_categories': len(CATEGORIES),
    'median_width': float(median_w),
    'median_height': float(median_h),
    'silhouette_pca': float(sil_pca),
    'silhouette_umap': float(sil_umap),
    'n_blurry': int(stats_df['is_blurry'].sum()),
    'n_bad_exposure': int(stats_df['bad_exposure'].sum())
}

with open(RESULTS_DIR / 'metrics.json', 'w') as f:
    json.dump(metrics, f, indent=2)

print("\n" + "="*70)
print("✅ EXPLORATION TERMINÉE")
print("="*70)
print(f"\nRésultats: {RESULTS_DIR}")
print("\n🎯 Prêt pour Notebook 5 (Deep Learning)")